In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, chi2
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

def selectkbest(indep_X, dep_Y, n):
    # Create a SelectKBest object with chi2 as the scoring function and n as the number of features to select
    test = SelectKBest(score_func=chi2, k=n)
    
    # Fit the SelectKBest object to the data
    fit1 = test.fit(indep_X, dep_Y)
    
    # Transform the data to select the top k features
    selectk_features = fit1.transform(indep_X)
    
    # Get the names of selected features
    selected_columns = indep_X.columns[fit1.get_support()]
    
    return selectk_features, selected_columns

def scalar(indep_X):
    sc = StandardScaler()
    indep_X = sc.fit_transform(indep_X)   
    return indep_X
    
def cm_prediction(classifier, indep_X, dep_Y):
    y_pred = classifier.predict(indep_X)
    cm = confusion_matrix(dep_Y, y_pred)
    Accuracy = accuracy_score(dep_Y, y_pred)
    report = classification_report(dep_Y, y_pred)
    return classifier, Accuracy, report, cm

def XGBClassifier(indep_X, dep_Y):
    from sklearn.model_selection import GridSearchCV
    from sklearn.neighbors import KNeighborsClassifier
    param_grid = {
        'n_neighbors': [ 50],  
        'weights': [distance'],  
        'algorithm': ['auto'] }             
    KNN= KNeighborsClassifier()
    classifier = GridSearchCV(estimator=KNN, param_grid=param_grid, cv=5, n_jobs=-1, verbose=3,refit=True, scoring='accuracy')
    classifier.fit(indep_X, dep_Y)
    print("Best parameters found:\n", classifier.best_params_)
    classifier, Accuracy, report, cm = cm_prediction(classifier,indep_X, dep_Y)
    print('Confusion Matrix:\n', cm)
    print('Classification Report:\n', report)
    from sklearn.metrics import roc_auc_score
    prob_predictions = classifier.predict_proba(indep_X)[:, 1]
    auc_score = roc_auc_score(dep_Y, prob_predictions)
    print('ROC AUC Score:', auc_score)
    return classifier, Accuracy, report, cm


def selectk_Classification(XGB): 
    dataframe = pd.DataFrame(index=['Accuracy'], columns=['XGBClassifier'])
    for number, idex in enumerate(dataframe.index):      
        dataframe['XGBClassifier'][idex] = XGB[number]       
    return dataframe

In [2]:
dataset1=pd.read_csv("EV_Pre-processed_Dataset.csv",index_col=None)

df2=dataset1

In [3]:
df2

Charging Duration (minutes)  Energy Consumed (kWh)  \
0                            112                     54   
1                             61                     31   
2                            102                     42   
3                             24                     14   
4                            116                     16   
..                           ...                    ...   
995                           13                     44   
996                          113                     43   
997                           13                     60   
998                           65                     12   
999                           34                     11   

     Charging Frequency (per week)  Initial Battery Level (%)  \
0                                7                         61   
1                                8                         24   
2                                2                         81   
3                                7                         70   
4                                2                         30   
..                             ...                        ...   
995                              6                         49   
996                              8                         50   
997                              8                         20   
998                              6                         32   
999                              4                         10   

     Final Battery Level (%)  Battery Health  Temperature (°C)  Humidity (%)  \
0                         65               1                25            79   
1                         40               1                19            58   
2                         55               1                37            27   
3                         73               1                36            44   
4                         76               1                33            32   
..                       ...             ...               ...           ...   
995                       38               1                28            78   
996                       87               1                33            35   
997                       61               1                28            78   
998                       51               1                37            24   
999                       58               1                25            23   

     Vehicle Age (years)  Battery Capacity (kWh)  ...  \
0                      6                      30  ...   
1                      4                      32  ...   
2                      3                      75  ...   
3                      9                      84  ...   
4                      2                      93  ...   
..                   ...                     ...  ...   
995                    7                      87  ...   
996                    8                      66  ...   
997                    4                      35  ...   
998                    1                      94  ...   
999                    1                      35  ...   

     Time Spent Driving (minutes)  Weekend Usage (%)  Is Regular Charger  \
0                              66                 60                   0   
1                              93                 80                   1   
2                             248                 34                   1   
3                              63                 64                   1   
4                              68                 58                   1   
..                            ...                ...                 ...   
995                            41                  4                   1   
996                           268                 75                   1   
997                           110                 62                   0   
998                           299                  0                   1   
999                            77

In [4]:
df2 = pd.get_dummies(df2,dtype=int,drop_first=True)

In [5]:
indep_X = df2.drop('Is Regular Charger', axis=1)
dep_Y = df2['Is Regular Charger']

In [6]:
# Get selected features and their names
selected_features, selected_columns = selectkbest(indep_X, dep_Y,5)

# Display the selected features shape
print("Selected Features Shape:", selected_features.shape)

# Display the names of the selected columns
print("Selected Feature Columns:")
print(selected_columns)

Selected Features Shape: (1000, 5)
Selected Feature Columns:
Index(['Charging Duration (minutes)', 'Initial Battery Level (%)',
       'Battery Capacity (kWh)', 'Maximum Range (km)', 'Charging Station ID'],
      dtype='object')


In [7]:
# Split and scale the dataset
indep_X = scalar(selected_features)

In [8]:
XGB=[]
classifier, Accuracy, report, cm = XGBClassifier(indep_X, dep_Y)
XGB.append(Accuracy)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters found:
 {'algorithm': 'auto', 'n_neighbors': 50, 'weights': 'distance'}
Confusion Matrix:
 [[475   0]
 [  0 525]]
Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00       475
           1       1.00      1.00      1.00       525

    accuracy                           1.00      1000
   macro avg       1.00      1.00      1.00      1000
weighted avg       1.00      1.00      1.00      1000

ROC AUC Score: 1.0


In [9]:
result = selectk_Classification(XGB)

C:\Users\swath\AppData\Local\Temp\ipykernel_12360\169966270.py:59: FutureWarning: ChainedAssignmentError: behaviour will change in pandas 3.0!
You are setting values through chained assignment. Currently this works in certain cases, but when using Copy-on-Write (which will become the default behaviour in pandas 3.0) this will never work to update the original DataFrame or Series, because the intermediate object on which we are setting values will behave as a copy.
A typical example is when you are setting values in a column of a DataFrame, like:

df["col"][row_indexer] = value

Use `df.loc[row_indexer, "col"] = values` instead, to perform the assignment in a single step and ensure this keeps updating the original `df`.

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

  dataframe['XGBClassifier'][idex] = XGB[number]


In [10]:
result
#k=5

XGBClassifier
Accuracy           1.0